# Import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# PD Setting


In [ ]:
# 显示所有列
pd.set_option('display.max_columns', None)
# 显示所有行
pd.set_option('display.max_rows', None)
#显示宽度
pd.set_option('display.width', 2000)

pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)

# Load Data

In [ ]:
#load data
train_path ="./data/train.csv"
test_path="./data/test.csv"
train_data = pd.read_csv(train_path)
test_data =pd.read_csv(test_path)
print('实验数据大小:',train_data.shape)
print('预测数据大小:',test_data.shape)

# EDA

info（）

* 行数（非空值计数）：显示每一列中有多少个非空值，这可以帮助你了解数据的完整性和是否存在缺失值。
* 列名：展示 DataFrame 中所有列的名字。
* 列类型（Dtype）：每列的数据类型（如 int64, float64, object 等），数据类型对于数据分析非常重要，因为不同类型的数据处理方式不同。
* 内存使用情况：显示该 DataFrame 在内存中的占用大小，这对于优化性能和资源管理特别有用，特别是当你在处理非常大的数据集时。

In [ ]:
train_data.info()

describe()


* count（计数）：非空值的数量。
* mean（平均数）：每列数值的平均值。
* std（标准差）：衡量数值的变异程度，即数值与平均值之间的偏离度。
* min（最小值）：每列中的最小数值。
* 25%（第一四分位数）：也称为下四分位数，表示有25%的数据小于等于这个数值。
* 50%（第二四分位数/中位数）：数据中间的值，即一半的数据小于该值，另一半大于该值。
* 75%（第三四分位数）：表示有75%的数据小于等于这个数值。
* max（最大值）：每列中的最大数值。

如果你的数据框中有非数值类型（例如对象或类别类型），你可能需要通过传递参数 include=['object', 'category'] 来查看这些类型列的描述性统计信息

In [ ]:
train_data.describe()

value_counts（）

用于计算分类变量中每个类别的出现频率（占比）的方法

发现数据集严重不平衡，可能需要考虑采用重采样技术或其他策略来平衡数据集。

In [ ]:
train_data['target'].value_counts(normalize=True)

head()

用于显示 DataFrame 的前几行数据的方法

In [ ]:
train_data.head(5)

isnull().sum()

用来表示数据中的缺失值

In [ ]:
missing_values = train_data.isna().sum()
# 或者
missing_values = train_data.isnull().sum()

# 查看缺失值
missing_values[missing_values > 0]


# 查看某列包含缺失值（NaN）的前几条记录
train_data[train_data['column_name'].isnull()].head()

.nunique()

用于返回对象中除 NA/null 值外的不同(unique)的非重复值的数量。

In [ ]:
train_data.nunique()

.unique()

查看某列里，所有不同的（唯一）值

In [ ]:
col_name = 'target'
train_data[col_name].unique()

.corr()

* 'pearson'：标准的皮尔逊相关系数。
* 'kendall'：Kendall Tau 相关系数。
* 'spearman'：Spearman 秩相关系数。

In [ ]:

CorrDf = pd.DataFrame()
CorrDf = train_data.corr(method='pearson', min_periods=1)

#只显示与 target 相关性较高的特征，从而使得图表更加简洁和易于理解
print(CorrDf['target'].sort_values())
high_corr_features = CorrDf['target'].abs() > 0.03 
corr_matrix_filtered = CorrDf.loc[high_corr_features, high_corr_features]

plt.figure(figsize=(10, 8))  
sns.heatmap(corr_matrix_filtered, annot=True, cmap='coolwarm', vmin=-1, vmax=1)

# 显示图表
plt.title('Correlation Matrix Heatmap')
plt.show()

# Data Process



* 数据清洗（Data Cleaning）：
  
处理缺失值：可以通过删除含有缺失值的记录、填充缺失值（如使用均值、中位数或众数填充）等方式进行。

去除重复值：识别并删除数据集中的重复记录。

纠正错误值：检查数据中的逻辑错误或异常值，并进行相应的修正。

* 数据转换（Data Transformation）：
  
标准化/归一化：将数值属性缩放到某个特定范围，比如0-1之间，或者将其标准化以拥有零均值和单位方差。

编码分类变量：如你之前提到的，可以使用LabelEncoder或OneHotEncoder对分类数据进行编码。

特征构造：基于现有特征创建新的有意义的特征。

* 数据集成（Data Integration）：
  
合并来自不同源的数据，确保一致性，解决实体识别问题，以及处理冗余和冲突的数据。

* 数据规约（Data Reduction）：
  
降维：通过主成分分析（PCA）、线性判别分析（LDA）等技术减少数据维度。

数量规约：使用参数模型或非参数模型来代替原始数据。

* 数据离散化（Data Discretization）：
  
将连续型数据转换为离散区间，便于某些算法处理。

* 数据分割（Data Splitting）：
  
将数据集划分为训练集、验证集和测试集，用于模型训练、调参和评估。

In [ ]:
# 整合数据，可以一起处理
full_data = pd.concat([train_data,test_data],ignore_index=True)

In [ ]:
# Data Cleaning

#用一个特定的常数来填充所有的缺失值
full_data.fillna(0, inplace=True) 

# 用特定字母
full_data['column_name'].fillna('U')

#使用上一个非缺失值来填充当前的缺失值。这种方法通常适用于时间序列数据
full_data.fillna(method='ffill', inplace=True)

#与前向填充相反，使用下一个非缺失值来填充当前的缺失值
full_data.fillna(method='bfill', inplace=True)

#均值/中位数/众数填充
# 使用均值填充
full_data['column_name'].fillna(full_data['column_name'].mean(), inplace=True)

# 使用中位数填充
full_data['column_name'].fillna(full_data['column_name'].median(), inplace=True)

# 使用众数填充
full_data['column_name'].fillna(full_data['column_name'].mode()[0], inplace=True)

#插值法（Interpolation） 特别适合于时间序列数据，可以根据已有的数据点估计缺失值
full_data.interpolate(method='linear', inplace=True)

#出现频率最高的值来填充所有的缺失值
full_data = full_data.fillna({'column_name': full_data['column_name'].value_counts().idxmax()})

#从 Cabin 列中提取每个客舱的第一个字符来表示甲板（Deck）信息，并创建一个新的列 Deck 来存储这些信息。
full_data['Deck'] = full_data['Cabin'].apply(lambda x: x[0] if x != 'Unknown' else 'Unknown')



In [ ]:
# Data Drop

# 删除所有 column_name 列中值为 'some_value' 的行。
full_data.drop(full_data[full_data['column_name'] == 'some_value'].index, inplace=True)

# 删除含有任何缺失值的行
full_data.dropna(inplace=True)

# 删除含有任何缺失值的列
full_data.dropna(axis=1, inplace=True)

# 删除指定列
full_data.drop('column_name', axis=1, inplace=True)
full_data.drop(['column_name1', 'column_name2'], axis=1, inplace=True)

#基于条件删除列
thresh = len(full_data) * 0.6  # 至少需要60%的非空值
full_data.dropna(axis=1, thresh=thresh, inplace=True)

In [ ]:
# Data Transformation
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# 避免污染到原始数据
full_data =train_data.copy() 
# 对每个分类特征应用标签编码
for column in full_data.columns:
    if(full_data[column].dtype =='object'):
        full_data[column] = le.fit_transform(full_data[column])

In [ ]:
# Data Discretization

# 对数变换 有助于减少数据中的偏斜(skewness)，特别是当存在大量小值和少量极大值时 使得数据分布更加接近正态分布
full_data['column_name'] = np.log1p(full_data['column_name'])

# 找出数值型特征（列），并计算这些特征的偏度（skewness）。然后，它会筛选出那些偏度绝对值大于1的特征，并准备对这些特征进行对数变换以减少数据偏斜
numeric_df = full_data.select_dtypes(['float64','int32','int64'])
numeric_cols = numeric_df.columns.tolist()
skewed_cols = full_data[numeric_cols].apply(lambda x: x.skew()).sort_values(ascending=False)
skewed_df = pd.DataFrame({'skew':skewed_cols})
skew_cols = skewed_df[skewed_df['skew'].abs()>1].index.tolist()
for col in skew_cols:
    # 避免对含有0或负数的列直接取对数，可以先加上一个最小正值
    min_val = full_data[col].min()
    if min_val <= 0:
        full_data[col] = full_data[col] - min_val + 1e-8  # 调整值域使所有值都大于0
    full_data[col] = np.log(full_data[col])


# 分箱（Binning）是一种将连续数据离散化的方法，可以减少异常值的影响，使模型更加稳定  
# 划分到5个等宽的区间（bins）
full_data['column_name'] = pd.cut(full_data['column_name'], bins=5, labels=False)


#按自己的需求划分区间
full_data.loc[full_data['column_name'] <= 7.91, 'column_name'] = 0
full_data.loc[(full_data['column_name'] > 7.91) & (full_data['column_name'] <= 14.454), 'column_name'] = 1
full_data.loc[(full_data['column_name'] > 14.454) & (full_data['column_name'] <= 31), 'column_name'] = 2
full_data.loc[full_data['column_name'] > 31, 'column_name'] = 3



In [ ]:
# Data Split
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42) 

Clean Text

In [ ]:
import re
import string
import nltk
import pandas as pd
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# retrieve english stop words
stop_words = set(stopwords.words('english'))

# Converts text to lower case
def convert_to_lowercase(text):
    if pd.isna(text):
        return text
    if isinstance(text, str):
        return text.lower()
    return text

# Remove all punctuation from the text
def remove_punctuation(text):
    if pd.isna(text):
        return text
    text = re.sub(f'[{string.punctuation}]', '', text)
    return text

# Removes all numbers from the text
def remove_numbers(text):
    if pd.isna(text):
        return text
    text = re.sub(r'\d+', '', text)
    return text

# Text segmentation, then remove the length of 2 or less and the single word and stop word
def remove_short_words_and_stop_words(text):
    if pd.isna(text):
        return text
    words = word_tokenize(text)
    words = [word for word in words if len(word) > 2 and word not in stop_words]
    cleaned_text = ' '.join(words)
    return cleaned_text

# Replace two or more consecutive Spaces with a single space
def remove_multiple_spaces(text):
    if pd.isna(text):
        return text
    cleaned_text = re.sub(r' {2,}', ' ', text)
    return cleaned_text

# Remove urls
def remove_urls(text):
    if pd.isna(text):
        return text
    url_pattern = r'(www.|http[s]?://)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_pattern, '', text)

# Remove hmtmls
def remove_html(text):
    if pd.isna(text):
        return text
    html_entities = r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});'
    return re.sub(html_entities, '', text)

# Remove @ and #
def remove_tags(text):
    if pd.isna(text):
        return text
    tag_pattern = r'@([a-z0-9]+)|#'
    return re.sub(tag_pattern, '', text)

def remove_emoji(text):
    if pd.isna(text):
        return text
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def preprocess_text(text):
    if pd.isna(text):
        return text
    cleaned_text = re.sub(r'[^a-zA-Z\d\s]+', '',text)
    word_list = []
    for each_word in cleaned_text.split(' '):
        word_list.append((each_word).lower())
    word_list = [
        WordNetLemmatizer().lemmatize(each_word.strip()) for each_word in word_list
        if each_word not in stop_words and each_word.strip() != ''
    ]
    return " ".join(word_list)



def clear_text(df, col):
    df[col] = df[col].apply(convert_to_lowercase)
    df[col] = df[col].apply(remove_urls)
    df[col] = df[col].apply(remove_html)
    df[col] = df[col].apply(remove_tags)
    df[col] = df[col].apply(remove_numbers)
    df[col] = df[col].apply(remove_short_words_and_stop_words)
    df[col] = df[col].apply(preprocess_text)
    df[col] = df[col].apply(remove_multiple_spaces) 
    # df[col] = df[col].apply(remove_emoji) 
    # df[col] = df[col].apply(correct_spellings)
    return df

# Model

## 常见的回归算法


scoring

* 分类任务：
'accuracy': 准确率，正确预测的比例。
'balanced_accuracy': 平衡准确率，适用于不平衡数据集。
'precision', 'precision_macro', 'precision_micro', 'precision_weighted', 'precision_samples': 精确率，不同参数对应不同的平均策略。
'recall', 'recall_macro', 'recall_micro', 'recall_weighted', 'recall_samples': 召回率，同样有不同的平均策略。
'f1', 'f1_macro', 'f1_micro', 'f1_weighted', 'f1_samples': F1分数，是精确率和召回率的调和平均数。
'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo': ROC曲线下面积，用于二分类或多分类。
* 回归任务：
'neg_mean_squared_error'(NMSE): 负均方误差，值越小越好。
'neg_mean_absolute_error'(NMAE): 负平均绝对误差。
'neg_mean_squared_log_error': 负均方对数误差。
'neg_median_absolute_error': 负中位数绝对误差。
'r2': R² (决定系数)，表示模型解释变异性的比例，值越大越好。
'explained_variance': 解释方差得分。
* 聚类任务：
'adjusted_rand_score': 调整后的兰德指数，用于比较两个聚类结果的一致性。
* 其他：
'neg_log_loss': 负对数损失，适用于概率预测。
'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_weighted', 'jaccard_samples': 杰卡德相似系数

In [ ]:
# %%skip

#导入机器学习算法库
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, KFold
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
# 设置 kfold，交叉采样法拆分数据集
kfold = KFold(n_splits=10)
lgb = LGBMRegressor(objective='regression', force_col_wise=True, random_state=42)
# xgb = XGBRegressor(objective='reg:linear', random_state=50, silent=True)
# 汇总不同模型算法
regressors = []
regressors.append(LinearRegression())
regressors.append(DecisionTreeRegressor())
regressors.append(RandomForestRegressor())
regressors.append(ExtraTreesRegressor())
regressors.append(GradientBoostingRegressor())
regressors.append(KNeighborsRegressor())
regressors.append(Ridge())
regressors.append(Lasso())
regressors.append(SVR())
# regressors.append(lgb)
# regressors.append(xgb)

cv_results=[]

# 示例代码：使用交叉验证评估每个模型
for regressor in regressors:
    results = cross_val_score(regressor, X_train, Y_train, cv=kfold, scoring='neg_mean_squared_error')
    cv_results.append(results)
    # print(f"{regressor.__class__.__name__}: Mean Squared Error: {-cv_results.mean()} (+/- {cv_results.std()})") 

cv_means=[]
cv_std=[]
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())
    
    
#汇总数据
cvResDf=pd.DataFrame({'cv_mean':cv_means,
                     'cv_std':cv_std,
                     'algorithm':['LR','DT','RandomForestRe','ExtraTreesRe',
                                  'GradientBoostingRe','KNN RE','Ridge','Lasso','SVR','lgb','xgb']})
#neg_mean_squared_error 数值越接近 0，表示模型效果越好
print(cvResDf)
''' 
    cv_mean    cv_std           algorithm
0 -0.018805  0.008046                  LR
1 -0.044421  0.012643                  DT
2 -0.019065  0.004311      RandomForestRe
3 -0.017231  0.005352        ExtraTreesRe
4 -0.015835  0.003744  GradientBoostingRe
5 -0.064388  0.010701              KNN RE
6 -0.018720  0.007841               Ridge
7 -0.063309  0.011553               Lasso
8 -0.052282  0.009065                 SVR
'''

In [ ]:
# 找到某一个合适的Model，超参数调优
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
X_train =np.array(X_train)
X_test =np.array(X_test)
print("是否存在任何 NaN:", np.isnan(X_train).any())
parameters = {'n_estimators': [10, 50, 100, 200, 300, 400, 500],
              'criterion': ['poisson', 'absolute_error', 'friedman_mse', 'squared_error'],
              'max_features': ['sqrt', 'log2']}
#n_estimators 模型中基础估计器（通常是决策树）的数量。
model =RandomForestRegressor()
grid_search = GridSearchCV(model, parameters, scoring='neg_mean_squared_error', n_jobs=4)

grid_search.fit(X_train, Y_train)
print('best parameters:',grid_search.best_params_)
print('best score:',grid_search.best_score_)
# 预测
y_pred_rf = grid_search.predict(X_test)


def save_data(y_pred,filename):
    submission=pd.DataFrame ( {
    "Id" : test_data [ "Id" ],
    "SalePrice" : y_pred.astype (int)
    })
    submission.to_csv ( filename+'_result.csv',index=False)
    
save_data(y_pred_rf,'random')



## 常见的分类算法

In [ ]:
#导入机器学习算法库
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold

#设置kfold，交叉采样法拆分数据集
kfold=StratifiedKFold(n_splits=5)

#汇总不同模型算法
classifiers=[]
classifiers.append(SVC())
classifiers.append(DecisionTreeClassifier())
classifiers.append(RandomForestClassifier())
classifiers.append(ExtraTreesClassifier())
classifiers.append(GradientBoostingClassifier())
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression())
classifiers.append(LinearDiscriminantAnalysis())

cv_results=[]
for classifier in classifiers:
    cv_results.append(cross_val_score(classifier,X_train,Y_train,
                                      scoring='accuracy',cv=kfold,n_jobs=-1))
    
cv_means=[]
cv_std=[]
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())
    
#汇总数据
cvResDf=pd.DataFrame({'cv_mean':cv_means,
                     'cv_std':cv_std,
                     'algorithm':['SVC','DecisionTreeCla','RandomForestCla','ExtraTreesCla',
                                  'GradientBoostingCla','KNN','LR','LinearDiscrimiAna']})

print(cvResDf)
''' 
  cv_mean    cv_std            algorithm
0  0.640864  0.011413                  SVC
1  0.789053  0.028307      DecisionTreeCla
2  0.781181  0.017948      RandomForestCla
3  0.767723  0.021185        ExtraTreesCla
4  0.826044  0.013216  GradientBoostingCla
5  0.795725  0.019425                  KNN
6  0.802473  0.008145                   LR
7  0.794614  0.016101    LinearDiscrimiAna
'''

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
print("是否存在任何 NaN:", np.isnan(X_train).any())
parameters ={'n_estimators':[10,50,100,200,300,400,500],'criterion':['log_loss','gini','entropy'],'max_features':['sqrt','log2']}
#n_estimators 模型中基础估计器（通常是决策树）的数量。
model =RandomForestClassifier()
grid_search =GridSearchCV(model,parameters,scoring='accuracy',n_jobs=4) #n_jobs 用于指定并行作业的数量

grid_search.fit(X_train, Y_train)
print('best parameters:',grid_search.best_params_)
print('best score:',grid_search.best_score_)
# 预测
y_pred_rf = grid_search.predict(X_test)

save_data(y_pred_rf,'random')
# 评估
# best score: 0.8204506936162   --- 0.75119

# Evaluation

In [ ]:
# Implement the evaluation metrics that we will use to assess our models

def accuracy(predicted_labels, true_labels):
    """
    Accuracy is correct predictions / all predicitons
    
    Args:
        predicted_labels (np.ndarray[int, 1]): the integer labels from the predictions. Uni-dimensional
        true_labels (np.ndarray[int, 1]): the integer labels from the gold standard. Uni-dimensional
    
    Returns:
        accuracy_value (double)
        
    """
    accuracy_value = 0.
    accuracy_value = predicted_labels[predicted_labels == true_labels].shape[0] / predicted_labels.shape[0]
    return accuracy_value

def precision(predicted_labels, true_labels):
    """
    Precision is True Positives / All Positives Predictions
    
    Args:
        predicted_labels (np.ndarray[int, 1]): the integer labels from the predictions. Uni-dimensional
        true_labels (np.ndarray[int, 1]): the integer labels from the gold standard. Uni-dimensional
    
    Returns:
        precision_value (double)
        
    """
    precision_value = 0.
    TP = np.sum((predicted_labels == 1) & (true_labels == 1))
    FP = np.sum((predicted_labels == 1) & (true_labels == 0))
    precision_value = TP / (TP + FP) if (TP + FP) > 0 else 0
    return precision_value

def recall(predicted_labels, true_labels):
    """
    Recall is True Positives / All Positive Labels
    
    Args:
        predicted_labels (np.ndarray[int, 1]): the integer labels from the predictions. Uni-dimensional
        true_labels (np.ndarray[int, 1]): the integer labels from the gold standard. Uni-dimensional
    
    Returns:
        recall_value (double)
        
    """
    recall_value = 0.
    TP = np.sum((predicted_labels == 1) & (true_labels == 1))
    FN = np.sum((predicted_labels == 0) & (true_labels == 1))
    recall_value = TP / (TP + FN) if (TP + FN) > 0 else 0
    return recall_value

def f1_score(predicted_labels, true_labels):
    """
    F1 score is the harmonic mean of precision and recall
    
    Args:
        predicted_labels (np.ndarray[int, 1]): the integer labels from the predictions. Uni-dimensional
        true_labels (np.ndarray[int, 1]): the integer labels from the gold standard. Uni-dimensional
    
    Returns:
        f1_score_value (double)
        
    """
    f1_score_value = 0.
    P = precision(predicted_labels, true_labels)
    R = recall(predicted_labels, true_labels)
    f1_score_value = 2 * P * R / (P + R) if (P + R) > 0 else 0
    return f1_score_value

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score, precision_score, recall_score

cm = confusion_matrix(yTrue, yPred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

f1 = f1_score(y_true, y_pred)
print("F1 Score:", f1)

precision = precision_score(y_true, y_pred)
print("Precision Score:", precision)

recall = recall_score(y_true, y_pred)
print("Recall Score:", recall)